Add a set of image-mask pairs to a Nora project

In [1]:
from src.config import config
import pandas as pd
import os
import numpy as np
import nibabel as nib
import sys
sys.path.append('/nfs/norasys/main/src/python')
from DPX_core import *

Loaded configuration for machine: nora


In [2]:
# get original files
img_dir = os.path.join(config["DATA_DIR"], "3D_val_npz")
gts_dir = os.path.join(config["DATA_DIR"], "3D_val_gt_interactive_seg")
current_dir = os.getcwd()


segfm3d_stats_bycase = pd.read_csv(os.path.join(current_dir, "ic_segmentation/notebooks/segfm3d_stats_bycase.csv"))
task_name = "MR_BraTS-T2f"  # "MR_BraTS-T2w" "CT_Aorta" #"CT_LungMask" # "" # "MR_WMH_T1"  
case_name_list = segfm3d_stats_bycase[segfm3d_stats_bycase["CaseName"].str.startswith(task_name)]["CaseName"].tolist()
print(len(case_name_list))
print(case_name_list)

import random
case_subset = random.sample(case_name_list, 1)
print(case_subset)

135
['MR_BraTS-T2f_bratsgli_0006', 'MR_BraTS-T2f_bratsgli_0009', 'MR_BraTS-T2f_bratsgli_0018', 'MR_BraTS-T2f_bratsgli_0029', 'MR_BraTS-T2f_bratsgli_0030', 'MR_BraTS-T2f_bratsgli_0038', 'MR_BraTS-T2f_bratsgli_0046', 'MR_BraTS-T2f_bratsgli_0049', 'MR_BraTS-T2f_bratsgli_0070', 'MR_BraTS-T2f_bratsgli_0075', 'MR_BraTS-T2f_bratsgli_0104', 'MR_BraTS-T2f_bratsgli_0107', 'MR_BraTS-T2f_bratsgli_0108', 'MR_BraTS-T2f_bratsgli_0121', 'MR_BraTS-T2f_bratsgli_0140', 'MR_BraTS-T2f_bratsgli_0151', 'MR_BraTS-T2f_bratsgli_0154', 'MR_BraTS-T2f_bratsgli_0159', 'MR_BraTS-T2f_bratsgli_0180', 'MR_BraTS-T2f_bratsgli_0199', 'MR_BraTS-T2f_bratsgli_0214', 'MR_BraTS-T2f_bratsgli_0215', 'MR_BraTS-T2f_bratsgli_0219', 'MR_BraTS-T2f_bratsgli_0223', 'MR_BraTS-T2f_bratsgli_0237', 'MR_BraTS-T2f_bratsgli_0238', 'MR_BraTS-T2f_bratsgli_0255', 'MR_BraTS-T2f_bratsgli_0256', 'MR_BraTS-T2f_bratsgli_0263', 'MR_BraTS-T2f_bratsgli_0264', 'MR_BraTS-T2f_bratsgli_0276', 'MR_BraTS-T2f_bratsgli_0279', 'MR_BraTS-T2f_bratsgli_0301', 'MR_B

In [ ]:
# creat niftis for images and gts
for case_filename in case_subset:
    print("Processing case: ", case_filename)


    img_path = os.path.join(img_dir, case_filename + ".npz")
    img_data = np.load(img_path)
    img = img_data["imgs"]
    img_spacing = img_data["spacing"]
    img_spacing = img_spacing[::-1]  # reverse spacing order
    gt_path = os.path.join(gts_dir, case_filename + ".npz")
    gt_data = np.load(gt_path)
    gt = gt_data["gts"]
    gt_spacing = gt_data["spacing"]
    gt_spacing = gt_spacing[::-1]  # reverse spacing order
    print("Image shape: ", img.shape)
    print("GT shape: ", gt.shape)

    # convert to nifty and save in nora project
    

    img_nifti = nib.Nifti1Image(img, affine=np.diag(np.append(img_spacing, 1)))
    nib.save(img_nifti, os.path.join(config["RESULTS_DIR"], case_filename + "_img.nii.gz"))
    gt_nifti = nib.Nifti1Image(gt, affine=np.diag(np.append(gt_spacing, 1)))
    nib.save(gt_nifti, os.path.join(config["RESULTS_DIR"], case_filename + "_gt.nii.gz"))


Processing case:  MR_BraTS-T2f_bratsgli_1071
Image shape:  (32, 218, 182)
GT shape:  (32, 218, 182)


In [14]:
# add to nora project 
# juypter session needs to be run from a project that has access to the destination project
project_name = "camaret___in_context_segmentation"

patient_pattern = f"(?<patients_id>{task_name})"
study_pattern = r"(?<studies_id>\w+)"
img_regex = f"{patient_pattern}_{study_pattern}_img\\.nii\\.gz"
gt_regex = f"{patient_pattern}_{study_pattern}_gt\\.nii\\.gz"

print(f"Importing image files for task: {task_name}")
!nora -p {project_name} --importfiles {config["RESULTS_DIR"]} '{img_regex}'

print(f"Importing ground truth masks for task: {task_name}")
!nora -p {project_name} --importfiles {config["RESULTS_DIR"]} '{gt_regex}' --addtag mask

"""
!nora -p {project_name} -a --patients_id {case_filename} {os.path.join(config["RESULTS_DIR"], case_filename + "_img.nii.gz")}
!nora -p {project_name} -a --patients_id {case_filename} {os.path.join(config["RESULTS_DIR"], case_filename + "_gt.nii.gz")} --addtag mask 
"""

Importing image files for task: MR_BraTS-T2f
searching in /nfs/data/nii/data1/Analysis/camaret___in_context_segmentation/ANALYSIS_20251122/results with regexp (?<patients_id>MR_BraTS-T2f)_(?<studies_id>\w+)_img\.nii\.gz
importing MR_BraTS-T2f#bratsgli_1262 file:/nfs/data/nii/data1/camaret___in_context_segmentation/MR_BraTS-T2f/bratsgli_1262/MR_BraTS-T2f_bratsgli_1262_img.nii.gz
importing MR_BraTS-T2f#bratsgli_1172 file:/nfs/data/nii/data1/camaret___in_context_segmentation/MR_BraTS-T2f/bratsgli_1172/MR_BraTS-T2f_bratsgli_1172_img.nii.gz
importing MR_BraTS-T2f#bratsgli_1096 file:/nfs/data/nii/data1/camaret___in_context_segmentation/MR_BraTS-T2f/bratsgli_1096/MR_BraTS-T2f_bratsgli_1096_img.nii.gz
importing MR_BraTS-T2f#bratsgli_1071 file:/nfs/data/nii/data1/camaret___in_context_segmentation/MR_BraTS-T2f/bratsgli_1071/MR_BraTS-T2f_bratsgli_1071_img.nii.gz
importing MR_BraTS-T2f#bratsgli_1033 file:/nfs/data/nii/data1/camaret___in_context_segmentation/MR_BraTS-T2f/bratsgli_1033/MR_BraTS-T2f_

'\n!nora -p {project_name} -a --patients_id {case_filename} {os.path.join(config["RESULTS_DIR"], case_filename + "_img.nii.gz")}\n!nora -p {project_name} -a --patients_id {case_filename} {os.path.join(config["RESULTS_DIR"], case_filename + "_gt.nii.gz")} --addtag mask \n'